In [1]:
import HelperFuncs as hfuncs
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.data_utils import Sequence
import h5py
import os

Using TensorFlow backend.


We're going to first try training a CNN on the individual images.
We will be using binary cross entropy across the 17 regions.

In [2]:
BATCH_SIZE = 20
FINAL_WIDTH = 400
FINAL_HEIGHT = 600
CHANNELS = 1
ZONES = 17

class Sequencer(Sequence):
    idx_dict={}
    
    def __init__(self,num_batches,bucket_name,mode="train"):
        self.num_batches = num_batches
        self.bucket_name = bucket_name
        self.mode = mode
        self.key_id, self.secret_key = hfuncs.GetAWSCredentials()
        self.mode = mode
        
        order = np.arange(self.num_batches)
        np.random.seed(0)
        np.random.shuffle(order)
        k = 0
        for k in range(len(order)):
            self.idx_dict[k]=order[k]

    def __len__(self):
        return self.num_batches
    def __getitem__(self,idx):
        #Get Client
        client = hfuncs.GetAWSClient(self.key_id,self.secret_key)
        bucket = client.Bucket(self.bucket_name)
        
        idx = self.idx_dict[idx] #Mix up order of the batches 
        
        path = "temp/{}/batch_{}.hdf5".format(self.mode,idx)
        key = "{}/batch_{}.hdf5".format(self.mode,idx)
        bucket.download_file(Key=key,Filename=path)
        
        f = h5py.File(path,"r")
        try:
            return f['/image'].value, f['/labels'].value
        finally:
            f.close()
            os.remove(path)
    def on_epoch_end(self):
        pass



Let's see how an existing architecture would do. 

In [12]:
from inception_resnet_v1 import InceptionResNetV1
from keras.layers import Input,Flatten,Dense,Concatenate,Dropout
from keras.models import Model
import keras
from datetime import datetime
from keras.callbacks import TensorBoard,EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from keras.optimizers import SGD
from keras import metrics
import os

def trainCNNRes(alpha,lr,momentum,pooling,dropout_rate,description=""):
    
    #ResNetv1
    input_img = Input(shape=(FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))
    incep = InceptionResNetV1(include_top=False,
                              weights=None,
                              input_tensor=input_img,
                             pooling=pooling,
                             alpha=alpha)
    #Apply dropout specified with specified rate
    last = incep.output
    last = Dropout(dropout_rate,seed=0)(last)
    
    #List of independent guesses for each zone
    output_nodes = []
    for i in range(ZONES):
        output_nodes.append(Dense(1,activation='sigmoid')(last))

    out = keras.layers.concatenate(output_nodes)
    
    #FInish model
    multi_label_model = Model(input_img, out)

    
    #optimizer
    nes=True
    sgd = SGD(lr,momentum=momentum,nesterov=nes)
    del incep
    multi_label_model.compile(optimizer=sgd,
                              metrics=[metrics.binary_accuracy],
                             loss= 'binary_crossentropy')
    ##Set up call-backs
    
    #Tensorboard
    x = datetime.today()
    stamp = "{}-{}-{}_{}:{}:{}_lr-{}_mom-{}_alpha-{}_pooling-{}_dropout-{}".format(x.year,x.month,
                                                         x.day,x.hour,x.minute,
                                                         x.second,lr,momentum,alpha,pooling,dropout_rate)
    tensorboard = TensorBoard(log_dir="logs/{}".format(stamp),histogram_freq=0,batch_size=BATCH_SIZE,
                              write_grads=False,write_images=True)
    
    #Early stopping callback
    estop = EarlyStopping(monitor='val_loss',min_delta=0.001,patience=4)
    
    #Model checkpoint
    check_point_dir = 'check_points/'
    if not os.path.isdir(check_point_dir):
        os.makedirs(check_point_dir)    
    chkpt = ModelCheckpoint(os.path.join(check_point_dir,"singleSGD_{epoch:02d}-{val_loss:.2f}.hdf5"),
                           monitor='val_loss',
                           verbose=1,
                           save_best_only=True)
    #Reduce learning rate on plateau
    reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                 factor=0.2,
                                 patience=2,
                                 min_lr=0.00005,
                                 cooldown = 5)
    
    ##Generators and fit
    
    #Bucket with clean data
    UPLOAD_BUCKET = 'cleandhsdata' #bucket where clean data was stored
    key_id, secret_key = hfuncs.GetAWSCredentials()
    client = hfuncs.GetAWSClient(key_id,secret_key)
    bucket = client.Bucket(UPLOAD_BUCKET)

    #Initialize train sequencer
    mode ="train"
    num_batches = sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1 #train,test,val root directories have their own keys
    train_seq = Sequencer(num_batches,UPLOAD_BUCKET,mode=mode)

    #Initialize validation sequencer
    mode = "val"
    num_batches = sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1 #train,test,val root directories have their own keys
    val_seq = Sequencer(num_batches,UPLOAD_BUCKET,mode=mode)

    
    
    hist = multi_label_model.fit_generator(train_seq,
                                           steps_per_epoch=3,
                                           epochs=100,
                                           validation_data = val_seq,
                                           validation_steps = 2,
                                           callbacks=[tensorboard,estop,chkpt,reduce_lr],
                                          use_multiprocessing =False,workers=1)
    return hist,multi_label_model

Hypermarameter Oprimization (Later)

In [ ]:
hist,model = trainCNNRes(alpha=1,lr=0.002,momentum=0.5,pooling='max',dropout_rate=0.25,description="")

Epoch 1/100
3/3 [==============================] - 83s - loss: 1.2296 - binary_accuracy: 0.7422 - val_loss: 0.4291 - val_binary_accuracy: 0.8603
Epoch 2/100
3/3 [==============================] - 33s - loss: 0.7144 - binary_accuracy: 0.8569 - val_loss: 0.3461 - val_binary_accuracy: 0.8794
Epoch 3/100
3/3 [==============================] - 34s - loss: 0.8256 - binary_accuracy: 0.8471 - val_loss: 0.3346 - val_binary_accuracy: 0.8794
Epoch 4/100
3/3 [==============================] - 34s - loss: 0.7945 - binary_accuracy: 0.8461 - val_loss: 0.3260 - val_binary_accuracy: 0.8794
Epoch 5/100
3/3 [==============================] - 33s - loss: 0.7765 - binary_accuracy: 0.8500 - val_loss: 0.3237 - val_binary_accuracy: 0.8912
Epoch 6/100
2/3 [===================>..........] - ETA: 8s - loss: 0.6764 - binary_accuracy: 0.8559 